In [11]:
import requests

def send_telegram(text, chat_ids):

    # Replace with your Telegram bot token
    bot_token = "7547680784:AAFK3Euccslel1Lu5ZpUUnOoG_Wvp3SXPzE"  # Example token

    url = f"https://api.telegram.org/bot{bot_token}/sendMessage"

    # Loop through each chat ID and send the message
    for chat_id in chat_ids:
        payload = {
            "chat_id": chat_id,
            "text": text
        }

        response = requests.post(url, json=payload)

        if response.status_code == 200:
            print(f"✅ Message sent to {chat_id} successfully!")
        else:
            print(f"❌ Failed to send to {chat_id}. Status code: {response.status_code}, Error: {response.text}")


# List of chat IDs to send the message
chat_ids = [
    "7759636739",   # Example Chat ID 1
    "1194971992"   # Example Chat ID 2    # Example Chat ID 3
]


In [33]:
import requests
import re

def escape_markdown_v2(text):
    """
    Escapes special characters for Telegram MarkdownV2 formatting.
    """
    escape_chars = r"_*[]()~`>#+-=|{}.!"
    return re.sub(r"([" + re.escape(escape_chars) + r"])", r"\\\1", text)



def send_telegram(text, chat_ids):
    """
    Sends a Telegram message with MarkdownV2 formatting to multiple chat IDs.
    """

    # Replace with your Telegram bot token
    bot_token = "7547680784:AAFK3Euccslel1Lu5ZpUUnOoG_Wvp3SXPzE"  # Example token

    url = f"https://api.telegram.org/bot{bot_token}/sendMessage"

    # Escape special characters for MarkdownV2
    formatted_text = escape_markdown_v2(text)

    # Loop through each chat ID and send the message
    for chat_id in chat_ids:
        payload = {
            "chat_id": chat_id,
            "text": formatted_text,
            "parse_mode": "MarkdownV2"   # Enable Markdown V2 formatting
        }

        response = requests.post(url, json=payload)

        if response.status_code == 200:
            print(f"✅ Message sent to {chat_id} successfully!")
        else:
            print(f"❌ Failed to send to {chat_id}. Status code: {response.status_code}, Error: {response.text}")


# List of chat IDs to send the message
chat_ids = [
    "7759636739",   # Example Chat ID 1
    "1194971992"    # Example Chat ID 2
]

In [36]:
import requests
from bs4 import BeautifulSoup
import json

# 👉 Replace with your OpenAI endpoint and key
openai_api_key = "63af3d0d40ad4a7fbdbd585fcda982d3"
openai_endpoint ="https://oai-dbme-seanbotplus-dev-wus.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-15-preview"

# URL of the website
#url = "https://www.moneycontrol.com/news/business/stocks/"
#url = "https://www.moneycontrol.com/news/tags/technical-analysis.html"
#url = "https://economictimes.indiatimes.com/news/economy/articlelist/1286551815.cms"
url = "https://economictimes.indiatimes.com/markets/stocks/news"

chat_ids = [
    "7759636739",   # Example Chat ID 1
    "1194971992"   # Example Chat ID 2
]
# Set headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36"
}
# Define the system prompt as a string
system_prompt = """
You are a financial data analyst. When provided with raw stock market data:
- Extract only the **stock names**.
- Classify each stock as either **bullish** or **bearish**.
- Briefly state the **reason** for the classification.
- Provide the **reason** in new line
- Ensure the response is concise, factual, and clear.
- Do NOT repeat the raw content—only summarize the stock names, classification, and reason.
- Ignore the stocks mentioned under the stocks to buy today, don't mention in the response
- create separate paragraph if any company has made any deal or got any project
"""

# Function to send text to OpenAI
def send_to_openai(text):
    payload = {
        "model": "gpt-4",  # Replace with your desired model
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": text}
        ],
        "temperature": 0.7,
        "max_tokens": 1500
    }

    openai_headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai_api_key}"
    }

    response = requests.post(openai_endpoint, headers=openai_headers, json=payload)

    if response.status_code == 200:
        print("\n✅ OpenAI Response:\n")
        print(response.json().get("choices", [{}])[0].get("message", {}).get("content", "No content returned"))
        #send_telegram(response.json().get("choices", [{}])[0].get("message", {}).get("content", "No content returned"))
        message = response.json().get("choices", [{}])[0].get("message", {}).get("content", "No content returned")
        send_telegram(message, chat_ids)
    else:
        print(f"❌ Failed to send data to OpenAI. Status code: {response.status_code}")
        print(response.text)

# Fetch the HTML content
response = requests.get(url, headers=headers)

if response.status_code == 200:
    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(response.text, "html.parser")

    # Extract section with id="mid"
    mid_section = soup.find(class_="clearfix main_container")

    if mid_section:
        # Extract only the text content
        text_content = mid_section.get_text(separator="\n", strip=True)

        # Send the extracted text to OpenAI
        send_to_openai(text_content)

    else:
        print("No section with id='mid' found.")
else:
    print(f"Failed to fetch HTML. Status code: {response.status_code}")



✅ OpenAI Response:

**Stock Names and Classifications:**

1. **KEI**
   - **Classification:** Bullish
   - **Reason:** Consistent performance and market recovery.

2. **GRSE**
   - **Classification:** Bullish
   - **Reason:** Strong performance in the defense sector.

3. **Infosys**
   - **Classification:** Bearish
   - **Reason:** Significant foreign outflows from the IT sector.

4. **IndusInd Bank**
   - **Classification:** Bearish
   - **Reason:** Notable decline in share price.

5. **Coal India**
   - **Classification:** Bullish
   - **Reason:** Strong performance and market recovery.

6. **Polycab**
   - **Classification:** Bullish
   - **Reason:** Market recovery and positive sector sentiment.

7. **Hexaware Technologies**
   - **Classification:** Bearish
   - **Reason:** Foreign outflows from the IT sector.

**Deals and Projects:**

**Garden Reach Shipbuilders & Engineers (GRSE)**
- Deal: Clearance of the 7,000-crore ATAGS Howitzer order.
- Project: Implementation of the Pinaka